# Import packages and set parameters

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Define methods

## Gap detection in DOS

In [ ]:
@jit(nopython=True)
def gap_detector(fermis, IDS, min_IDS_width = 0.001, min_energy_width = 0.1):
    gaps = np.zeros((0,2),dtype=np.float64)
    
    i = 0
    while i <= IDS.size:

        j = 1
        while abs(IDS[i+j]-IDS[i]) < min_IDS_width:
            if j+i == IDS.size-1:
                break
            else:
                j +=1
        else:
            if abs(fermis[i+j]-fermis[i]) > min_energy_width:
                gaps = np.append(gaps,np.array([[fermis[i],fermis[i+j]]]),axis=0)
                i += j
                continue
            else:
                i +=1
                continue
        
        return gaps

## Chern coefficients

In [ ]:
def n_u1t1(theta,Ch_u1t1,Ch_u1u2t1t2):
    return Ch_u1t1 + theta * Ch_u1u2t1t2

def n_u2t2(theta,Ch_u2t2,Ch_u1u2t1t2):
    return Ch_u2t2 + theta * Ch_u1u2t1t2

def n_(theta,Ch_,Ch_u1t1,Ch_u2t2,Ch_u1u2t1t2):
    return Ch_ - theta * Ch_u1t1 - theta * Ch_u2t2 - theta**2 * Ch_u1u2t1t2

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if data.at[0,'texture'] != "sdw":
        continue
    
    if "n_fermi" in data:
        df = pd.concat([df,data])
    if "n_moments" in data:
        df = pd.concat([df,data])
    if "tau1" in data:
        
        Chern_numbers = np.load(file+"/chern_numbers_tau1tau2u1u2.npy",allow_pickle=True)[()]

        for label in Chern_numbers:
            if label=="":
                data["IDS"]=Chern_numbers[label].real
            else:
                data[label]=Chern_numbers[label].real
        df = pd.concat([df,data])

display(df.set_index('key').sort_index(ascending=False))

## Select data

In [ ]:
key_spec, key_fermi_l, key_fermi_r = '1746356686', '1746954269', '1747011839' # m=0 s=0 theta_l=0.175 theta_r=0.438

gap_keys = ['1746717349','1746717698','1746718049','1746718402']

## Import file and extract data

In [ ]:
# parameter
size_l,  size_r  = df.set_index('key').at[key_fermi_l,'system_sizes'], df.set_index('key').at[key_fermi_r,'system_sizes']
theta_l, theta_r = df.set_index('key').at[key_fermi_l,'q'],            df.set_index('key').at[key_fermi_r,'q']

sys_size         = df.set_index('key').at[key_spec,'system_sizes']
t,       m       = df.set_index('key').at[key_spec,'t'],               df.set_index('key').at[key_spec,'m']
shift,   mag     = df.set_index('key').at[key_spec,'shift'],           df.set_index('key').at[key_spec,'mag']

n_energies, n_moments, n_random_states = df.set_index('key').at[key_spec,'n_energies'], df.set_index('key').at[key_spec,'n_moments'], df.set_index('key').at[key_spec,'n_random_states'],
scale, epsilon = 12, 0.01,

n_fermi_l, min_fermi_l, max_fermi_l = df.set_index('key').at[key_fermi_l,'n_fermi'],  df.set_index('key').at[key_fermi_l,'min_fermi'], df.set_index('key').at[key_fermi_l,'max_fermi']
n_fermi_r, min_fermi_r, max_fermi_r = df.set_index('key').at[key_fermi_r,'n_fermi'],  df.set_index('key').at[key_fermi_r,'min_fermi'], df.set_index('key').at[key_fermi_r,'max_fermi']

# Data
outdir_fermi_l = './data/' + key_fermi_l
outdir_fermi_r = './data/' + key_fermi_r
outdir_spec    = './data/' + key_spec

# DOS
fermis_l = np.linspace(min_fermi_l,max_fermi_l,n_fermi_l)
fermis_r = np.linspace(min_fermi_r,max_fermi_r,n_fermi_r)

spectrum_l = np.load(outdir_fermi_l+"/spectrum.npy")
spectrum_r = np.load(outdir_fermi_r+"/spectrum.npy")

# Chern numbers
# New Convention: tau1 = -t1 and order u1u2t1t2
# u1u2         -> u1u2
# tau1u1       -> (-1)^2 u1t1
# tau2u1       -> (-1)^2 u1t2
# tau1u2       -> (-1)^2 u2t1
# tau2u2       -> (-1)^2 u2t2
# tau1tau2     -> (-1)^2 t1t2
# tau1tau2u1u2 -> (-1)^2 u1u2t1t2

ch_0_l = np.array([])
for f in fermis_l:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum_l])
    ch_0_l = np.append(ch_0_l,[sum(occ)/size_l**2])

ch_u1u2_l     = np.load(outdir_fermi_l+"/ch_u1u2.npy")
ch_u1t1_l     = np.load(outdir_fermi_l+"/ch_tau1u1.npy")
ch_u1t2_l     = np.load(outdir_fermi_l+"/ch_tau2u1.npy")
ch_u2t1_l     = np.load(outdir_fermi_l+"/ch_tau1u2.npy")
ch_u2t2_l     = np.load(outdir_fermi_l+"/ch_tau2u2.npy")
ch_t1t2_l     = np.load(outdir_fermi_l+"/ch_tau1tau2.npy")
ch_u1u2t1t2_l = np.load(outdir_fermi_l+"/ch_tau1tau2u1u2.npy")

n_0_l        = n_(theta_l,ch_0_l,ch_u1t1_l,ch_u2t2_l,ch_u1u2t1t2_l) # _
n_u1u2_l     = ch_u1u2_l                                            # u1u2
n_u1t1_l     = n_u1t1(theta_l,ch_u1t1_l,ch_u1u2t1t2_l)              # u1t1
n_u1t2_l     = ch_u1t2_l                                            # u1t2
n_u2t1_l     = ch_u2t1_l                                            # u2t1
n_u2t2_l     = n_u2t2(theta_l,ch_u2t2_l,ch_u1u2t1t2_l)              # u2t2
n_t1t2_l     = ch_t1t2_l                                            # t1t2
n_u1u2t1t2_l = ch_u1u2t1t2_l                                        # u1u2t1t2                                                          

ch_0_r = np.array([])
for f in fermis_r:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum_r])
    ch_0_r = np.append(ch_0_r,[sum(occ)/size_r**2])

ch_u1u2_r     = np.load(outdir_fermi_r+"/ch_u1u2.npy")
ch_u1t1_r     = np.load(outdir_fermi_r+"/ch_tau1u1.npy")
ch_u1t2_r     = np.load(outdir_fermi_r+"/ch_tau2u1.npy")
ch_u2t1_r     = np.load(outdir_fermi_r+"/ch_tau1u2.npy")
ch_u2t2_r     = np.load(outdir_fermi_r+"/ch_tau2u2.npy")
ch_t1t2_r     = np.load(outdir_fermi_r+"/ch_tau1tau2.npy")
ch_u1u2t1t2_r = np.load(outdir_fermi_r+"/ch_tau1tau2u1u2.npy")

n_0_r        = n_(theta_r,ch_0_r,ch_u1t1_r,ch_u2t2_r,ch_u1u2t1t2_r) # _
n_u1u2_r     = ch_u1u2_r                                            # u1u2
n_u1t1_r     = n_u1t1(theta_r,ch_u1t1_r,ch_u1u2t1t2_r)              # u1t1
n_u1t2_r     = ch_u1t2_r                                            # u1t2
n_u2t1_r     = ch_u2t1_r                                            # u2t1
n_u2t2_r     = n_u2t2(theta_r,ch_u2t2_r,ch_u1u2t1t2_r)              # u2t2
n_t1t2_r     = ch_t1t2_r                                            # t1t2
n_u1u2t1t2_r = ch_u1u2t1t2_r                                        # u1u2t1t2 
    
# data
qs = np.array([i/sys_size for i in range(1,math.floor(sys_size/2))])

emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

n_q = len(qs)
dos = []
for i in range(n_q):
    dos.append( np.load('./data/'+key_spec+'/dos_'+str(i).zfill(4)+'.npy') )
dos = np.array(dos)

Emin = np.amin(emesh)
Emax = np.amax(emesh)

phimin = np.amin(qs)
phimax = np.amax(qs)

# gap markers
fermis = np.zeros(len(gap_keys))
thetas = np.zeros(len(gap_keys))
for i,k in enumerate(gap_keys):
    fermis[i] = df.set_index('key').at[k,'fermi']
    thetas[i] = df.set_index('key').at[k,'q']

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'cmu serif'

tfs    = 24 #title font size
lfs    = 24 #label font size
fs     = 22 #font size
cbarfs = 18 #colorbar font size
afs    = 24 #annotation font size
gfs    = 18 #gap font size

# scatter size
s = 2

linthresh = 0.07 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale)

# DoS + Chern

In [ ]:
gaps_l = gap_detector(fermis_l,ch_0_l,0.001,0.14)
gaps_r = gap_detector(fermis_r,ch_0_r,0.001,0.14)
print(gaps_l.shape,gaps_r.shape)
# print(gaps_l)
# print([sum(g)/2 for g in gaps_l])

In [ ]:
x_ticks0 = np.arange(0, 3, 1)
x_ticks1 = np.arange(0, 1, 1)
x_ticks2 = np.arange(0, 1, 1)
x_ticks3 = np.arange(-10, 10, 5)
x_ticks4 = np.arange(0.1, 0.5, 0.1)  
x_ticks5 = np.arange(-2, 4, 2)
x_ticks6 = np.arange(0, 1, 1)
x_ticks7 = np.arange(0, 1, 2)
x_ticks8 = np.arange(0, 15, 5)
y_ticks = np.arange(-10, 6, 2.5)
    
fig = plt.figure()
gs = fig.add_gridspec(1,30, hspace=0,wspace=0)
ax0 = plt.subplot(gs.new_subplotspec((0, 0), colspan=2))
ax1 = plt.subplot(gs.new_subplotspec((0, 2), colspan=2),sharey=ax0)
ax2 = plt.subplot(gs.new_subplotspec((0, 4), colspan=2),sharey=ax0)
ax3 = plt.subplot(gs.new_subplotspec((0, 6), colspan=5),sharey=ax0)
ax4 = plt.subplot(gs.new_subplotspec((0, 11), colspan=8),sharey=ax0)
ax5 = plt.subplot(gs.new_subplotspec((0, 19), colspan=2),sharey=ax0)
ax6 = plt.subplot(gs.new_subplotspec((0, 21), colspan=2),sharey=ax0)
ax7 = plt.subplot(gs.new_subplotspec((0, 23), colspan=2),sharey=ax0)
ax8 = plt.subplot(gs.new_subplotspec((0, 25), colspan=5),sharey=ax0)

ax0.scatter(n_0_l,fermis_l,s)
ax0.set_xticks(x_ticks0)
ax0.xaxis.set_minor_locator(MultipleLocator(1))
ax0.grid(True,which='minor',color='lightgray',axis='x')
ax0.set_xlim([-1,3])
ax0.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',labelpad=0.0,fontsize=lfs)
ax0.set_title(r'$n_\emptyset$',fontsize=tfs,y=1.01)

ax1.scatter(n_u1u2_l.real,fermis_l,s)
ax1.set_xticks(x_ticks1)
ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.grid(True,which='minor',color='lightgray',axis='x')
ax1.set_xlim([-2,2])
ax1.set_title(r'$n_{u_1 u_2}$',fontsize=tfs,y=1.005)

ax2.scatter(n_t1t2_l.real,fermis_l,s)
ax2.set_xticks(x_ticks2)
ax2.xaxis.set_minor_locator(MultipleLocator(1))
ax2.grid(True,which='minor',color='lightgray',axis='x')
ax2.set_xlim([-2,2])
ax2.set_title(r'$n_{t_1 t_2}$',fontsize=tfs,y=1.005)

ax3.scatter(n_u1u2t1t2_l.real,fermis_l,s)
ax3.set_xticks(x_ticks3)
ax3.xaxis.set_minor_locator(MultipleLocator(1))
ax3.grid(True,which='minor',color='lightgray',axis='x')
ax3.set_xlim([-15,10])
ax3.set_title(r'$n_{u_1 u_2 t_1 t_2}$',fontsize=tfs,y=1.005)

ax4.imshow(dos.T, aspect='auto',norm=norm, extent=(phimin,phimax, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax4.set_xticks(x_ticks4)
ax4.set_xlim([phimin,phimax])
ax4.xaxis.set_minor_locator(MultipleLocator(0.05))
ax4.set_title(r"DOS",fontsize=tfs,y=1.005)
ax4.set_xlabel(r"$\vartheta$",fontsize=lfs)

ax4.axvline(x = theta_l, color = 'red', label = r'$\theta_l$')
ax4.annotate(r'$\vartheta=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(theta_l*size_l,size_l),(theta_l+0.01,5.3),color='red',fontsize=afs)
ax4.axvline(x = theta_r, color = 'green', label = r'$\theta_r$')
ax4.annotate(r'$\vartheta=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(theta_r*size_r,size_r),(theta_r-0.12,-8.2),color='green',fontsize=afs)

ax4.annotate(r'$U_{-1}$',(0.01,4.5),color='black',fontsize=gfs)
ax4.annotate(r'$U_{-2}$',(0.01,3.9),color='black',fontsize=gfs)

ax4.annotate(r'$L_1$',(0.01,-6.9),color='black',fontsize=gfs)
ax4.annotate(r'$L_2$',(0.01,-5.65),color='black',fontsize=gfs)
ax4.annotate(r'$L_3$',(0.01,-4.95),color='black',fontsize=gfs)
ax4.annotate(r'$L_4$',(0.01,-4.15),color='black',fontsize=gfs)

ax4.annotate(r'$L_1$',(0.45,-3.6),color='black',fontsize=gfs)
ax4.annotate(r'$L_{NC}$',(0.445,-2.25),color='black',fontsize=gfs)

ax5.scatter(n_0_r,fermis_r,s)
ax5.set_xticks(x_ticks5)
ax5.xaxis.set_minor_locator(MultipleLocator(1))
ax5.grid(True,which='minor',color='lightgray',axis='x')
ax5.set_xlim([-3,3])
ax5.set_title(r'$n_\emptyset$',fontsize=tfs,y=1.01)

ax6.scatter(n_u1u2_r.real,fermis_r,s)
ax6.set_xticks(x_ticks6)
ax6.xaxis.set_minor_locator(MultipleLocator(1))
ax6.grid(True,which='minor',color='lightgray',axis='x')
ax6.set_xlim([-2,2])
ax6.set_title(r'$n_{u_1 u_2}$',fontsize=tfs,y=1.005)

ax7.scatter(n_t1t2_r.real,fermis_r,s)
ax7.set_xticks(x_ticks7)
ax7.xaxis.set_minor_locator(MultipleLocator(1))
ax7.grid(True,which='minor',color='lightgray',axis='x')
ax7.set_xlim([-2,2])
ax7.set_title(r'$n_{t_1 t_2}$',fontsize=tfs,y=1.005)

ax8.scatter(n_u1u2t1t2_r.real,fermis_r,s)
ax8.set_xticks(x_ticks8)
ax8.xaxis.set_minor_locator(MultipleLocator(1))
ax8.grid(True,which='minor',color='lightgray',axis='x')
ax8.set_xlim([-5,15])
ax8.set_title(r'$n_{u_1 u_2 t_1 t_2}$',fontsize=tfs,y=1.005)
ax8.yaxis.set_label_position("right")
ax8.get_yaxis().set_visible(False)

for ax in fig.get_axes():
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.set_yticks(y_ticks)
    ax.set_ylim([-9,6])
    
for ax in fig.get_axes()[0:4]:
    ax.grid(True,color='gray',axis='x')
    #ax.grid(True,which='minor',color='lightgray')
    
    for g in gaps_l:
        ax.fill_between(np.arange(-15,15,0.01),g[0],g[1], facecolor='gray', alpha=0.5)
        
for g in gaps_l:
    ax4.fill_between(np.arange(0.0,theta_l,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
for g in gaps_r:
    ax4.fill_between(np.arange(theta_r,0.51,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
    
for ax in fig.get_axes()[5:9]:
    ax.yaxis.tick_right()
    ax.grid(True,color='gray',axis='x')
    #ax.grid(True,which='minor',color='lightgray')
    
    for g in gaps_r:
        ax.fill_between(np.arange(-20,15,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
        
for ax in fig.get_axes()[1:]:
    ax.yaxis.tick_right()
    
# attaching gap labels
gaps = np.array([(thetas[i],fermis[i]) for i in range(len(gap_keys))]) # s=0,s=pi , m=0.0

gaps_col = cm.plasma(np.linspace(0,0.5,len(gaps)))
ax4.scatter(gaps[:,0],gaps[:,1],c=gaps_col,s=80,zorder=3)

plt.savefig("./Plots/sdw_Chernhierarchy_kpm.png", dpi=150, bbox_inches = 'tight')